In [ ]:
import plotly.graph_objects as go
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

pio.renderers.default = 'colab'
root_dir = '/home/hunteryt/notebooks/Capstone_Project/south_dist_chunked_10000to47495_dx5_fs100_bpf14to35Hz'

## Importing and cleaning data

In [ ]:
# import in clustering results
df = pd.read_csv('../data/eighty_cluster_results.csv')

In [ ]:
# get approximate likelihood of 
def get_confidence_proportion(conf_level):
    """
    Based on Confidence level return approximate confidence proportion
    """
    if conf_level == 3:
        return 1
    elif conf_level == 2:
        return 0.67
    elif conf_level == 1:
        return 0.33
    else:
        return 0

df['confidence_proportion'] = df.signal_confidence.apply(get_confidence_proportion)

In [ ]:
# Loading images of data
class DASImageDataset(Dataset):
    def __init__(self, root_dir, transform=transforms.ToTensor()):
        self.root_dir = root_dir
        self.files = os.listdir(self.root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img_file =(os.path.join(self.root_dir, self.files[index]))
        img = Image.open(img_file)
        
        if self.transform:
            img = self.transform(img)
        
        return img, self.files[index]
    
orig_transform = transforms.Compose([transforms.ToTensor()])
orig_images = DASImageDataset(root_dir=root_dir, transform=orig_transform)

## Heatmap Visualization

In [ ]:
def df_to_plotly(df):
    return {'z': df.confidence_proportion.tolist(),
            'x': df.time_start.str.replace(' ', 'T').tolist(),
            'y': df.distance_start.tolist()}

In [ ]:
fig = go.Figure(data=go.Heatmap(df_to_plotly(df)))
fig.update_layout(
    title='Fin Whale Vocalizations Heatmap',
    xaxis_title='Time',
    yaxis_title='Distance from shore (m)',
    legend_title='Confidence Level',
    hovermode='closest')